In [1]:
import numpy as np
import pandas as pd

# Importing Wine Data and adding columns' names
df = pd.read_csv("wine.data", header=None, 
                 names=['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash ', 'Magnesium', 'Total phenols',
                        'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue',
                        'OD280/OD315 of diluted wines', 'Proline'])
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
# Replace if NaN

df = df.dropna()
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
# Number of Rows and Columns
df.shape

(178, 14)

In [15]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

# Selecting Data
wineAttr = df.loc[:, df.columns != 'Class'].astype(float)
wineClass = df["Class"]

# Scaling Data
wineAttrScaled = scale.fit_transform(wineAttr[wineAttr.columns].values)

wineAttrScaled.shape

(178, 13)

In [25]:
from sklearn.model_selection import cross_val_score, train_test_split

# Separate data por train and test unscaled
Wine_train, Wine_test, Class_train, Class_test = train_test_split(wineAttr, wineClass, test_size=0.2, random_state=0)



In [26]:
#Multivariate Regression

from sklearn.linear_model import TheilSenRegressor

multi = TheilSenRegressor(random_state=0).fit(Wine_train, Class_train)

multi.score(Wine_test, Class_test) 

0.7312571300764876

In [27]:
# Check with Cross Validation

scores = cross_val_score(multi, wineAttr, wineClass, cv=10)

scores.mean()

0.06272541310529027

In [28]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(Wine_train, Class_train)

nb.score(Wine_test, Class_test) 

0.8611111111111112

In [29]:
# Check with Cross Validation

scores = cross_val_score(nb, wineAttr, wineClass, cv=10)

scores.mean()

0.8507675438596491

In [33]:
# Separate data por train and test scaled
Wine_train, Wine_test, Class_train, Class_test = train_test_split(wineAttrScaled, wineClass, test_size=0.2, random_state=0)

In [34]:
#SVM

from sklearn import svm

svc = svm.SVC(kernel='poly',degree=3, C=1, gamma='auto').fit(Wine_train, Class_train)

svc.score(Wine_test, Class_test) 

0.9444444444444444

In [35]:
# Check with Cross Validation

scores = cross_val_score(svc, wineAttrScaled, wineClass, cv=10)

scores.mean()

0.9502579979360165

In [39]:
#KNN

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(10)
knn.fit(Wine_train, Class_train)
knn.score(Wine_test, Class_test) 


1.0

In [40]:
# Check with Cross Validation

scores = cross_val_score(knn, wineAttrScaled, wineClass, cv=10)

scores.mean()

0.9666322669418644

In [61]:
# Machine learning
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(7, activation='relu', input_shape=(13,)))
model.add(Dropout(0.2))
model.add(Dense(6, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))


In [62]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [63]:
model.fit(Wine_train, Class_train,
          batch_size=10,
          epochs=150,
          verbose=2,
          validation_data=(Wine_test, Class_test))



Train on 142 samples, validate on 36 samples
Epoch 1/150
 - 2s - loss: 1.6099 - acc: 0.1972 - val_loss: 1.5737 - val_acc: 0.1111
Epoch 2/150
 - 0s - loss: 1.5562 - acc: 0.3099 - val_loss: 1.5148 - val_acc: 0.1389
Epoch 3/150
 - 0s - loss: 1.4623 - acc: 0.2887 - val_loss: 1.4709 - val_acc: 0.1667
Epoch 4/150
 - 0s - loss: 1.4212 - acc: 0.3521 - val_loss: 1.4338 - val_acc: 0.1944
Epoch 5/150
 - 0s - loss: 1.3912 - acc: 0.3803 - val_loss: 1.4004 - val_acc: 0.2500
Epoch 6/150
 - 0s - loss: 1.3706 - acc: 0.3803 - val_loss: 1.3651 - val_acc: 0.3333
Epoch 7/150
 - 0s - loss: 1.2809 - acc: 0.5070 - val_loss: 1.3327 - val_acc: 0.3611
Epoch 8/150
 - 0s - loss: 1.2544 - acc: 0.5070 - val_loss: 1.3000 - val_acc: 0.4167
Epoch 9/150
 - 0s - loss: 1.2488 - acc: 0.4648 - val_loss: 1.2638 - val_acc: 0.5278
Epoch 10/150
 - 0s - loss: 1.1977 - acc: 0.5282 - val_loss: 1.2304 - val_acc: 0.5833
Epoch 11/150
 - 0s - loss: 1.1314 - acc: 0.5634 - val_loss: 1.1966 - val_acc: 0.6389
Epoch 12/150
 - 0s - loss: 1.

Epoch 97/150
 - 0s - loss: 0.1993 - acc: 0.9437 - val_loss: 0.0813 - val_acc: 0.9722
Epoch 98/150
 - 0s - loss: 0.2100 - acc: 0.9648 - val_loss: 0.0812 - val_acc: 0.9722
Epoch 99/150
 - 0s - loss: 0.1922 - acc: 0.9366 - val_loss: 0.0809 - val_acc: 0.9722
Epoch 100/150
 - 0s - loss: 0.2276 - acc: 0.9437 - val_loss: 0.0815 - val_acc: 0.9722
Epoch 101/150
 - 0s - loss: 0.1554 - acc: 0.9648 - val_loss: 0.0804 - val_acc: 0.9722
Epoch 102/150
 - 0s - loss: 0.1839 - acc: 0.9437 - val_loss: 0.0788 - val_acc: 0.9722
Epoch 103/150
 - 0s - loss: 0.2304 - acc: 0.9366 - val_loss: 0.0750 - val_acc: 0.9722
Epoch 104/150
 - 0s - loss: 0.1389 - acc: 0.9577 - val_loss: 0.0737 - val_acc: 0.9722
Epoch 105/150
 - 0s - loss: 0.1761 - acc: 0.9648 - val_loss: 0.0724 - val_acc: 0.9722
Epoch 106/150
 - 0s - loss: 0.2458 - acc: 0.9225 - val_loss: 0.0704 - val_acc: 0.9722
Epoch 107/150
 - 0s - loss: 0.1707 - acc: 0.9507 - val_loss: 0.0676 - val_acc: 0.9722
Epoch 108/150
 - 0s - loss: 0.2048 - acc: 0.9507 - val_lo